---

# Customer Segmentation Analysis: E-commerce Marketing Strategy

### Leveraging K-Means Clustering for Targeted Customer Insights

---

##  Project Overview

In this project, I perform a comprehensive customer segmentation analysis for a UK-based online retail company. By applying unsupervised machine learning techniques to transactional data, I aim to identify distinct customer groups based on their purchasing behavior.

Understanding customer segments allows businesses to move from a "one-size-fits-all" marketing approach to personalized strategies that improve customer retention, increase average order value (AOV), and optimize marketing spend.

##  Dataset Information

The dataset used is the **Online Retail II** dataset from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/502/online+retail+ii).

* **Timeline:** Transactions occurring between 01/12/2009 and 09/12/2011.
* **Context:** The company primarily sells unique all-occasion gift-ware. Many customers are wholesalers.
* **Key Statistics:** * **Instances:** 525,461
* **Features:** Invoice number, Stock code, Description, Quantity, Invoice date, Unit price, Customer ID, and Country.



##  Business Objectives

1. **Data Engineering:** Transform raw transactional data into a customer-centric **RFM (Recency, Frequency, Monetary)** framework.
2. **Customer Understanding:** Perform Exploratory Data Analysis (EDA) to find trends in sales, geography, and top-selling products.
3. **Machine Learning:** Implement a **K-Means Clustering** algorithm to segment customers.
4. **Strategic Recommendations:** Interpret the clusters to provide actionable business recommendations for marketing and sales teams.

## Technical Workflow

1. **Data Cleaning:** Handling missing values (specifically Customer IDs) and removing cancellations/returns to ensure data integrity.
2. **RFM Feature Engineering:**
* **Recency:** Days since the last purchase.
* **Frequency:** Total number of purchases.
* **Monetary Value:** Total revenue generated by the customer.


3. **Pre-processing:** Scaling features using `StandardScaler` to handle the variance in RFM metrics and addressing skewness.
4. **Optimal Cluster Selection:** Utilizing the **Elbow Method** and **Silhouette Score** to determine the most effective number of clusters ().
5. **Cluster Profiling:** Visualizing clusters using 3D scatter plots and snake plots to define "Champions," "At-Risk," and "New Customers."

---

### Key Skills Demonstrated:

`Python` | `Pandas` | `Scikit-Learn` | `K-Means Clustering` | `RFM Analysis` | `Data Visualization (Matplotlib/Seaborn)` | `Exploratory Data Analysis (EDA)`



# 1. Import Necessary Packages 


In [4]:
import pandas as pd
# This turns the warning off globally
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples




# 2. Dataset Injestion 


In [ ]:
import pandas as pd
import zipfile
import urllib.request
import io

# Download the ZIP file
url = 'https://archive.ics.uci.edu/static/public/502/online+retail+ii.zip'
response = urllib.request.urlopen(url)
zip_data = io.BytesIO(response.read())

# Extract and read the Excel file from the ZIP
with zipfile.ZipFile(zip_data) as zip_ref:
    # List files in the archive to find the Excel file
    file_list = zip_ref.namelist()
    print("Files in archive:", file_list)
    
    # Read the first Excel file (adjust the filename if needed)
    excel_file = [f for f in file_list if f.endswith('.xlsx')][0]
    with zip_ref.open(excel_file) as excel_data:
        df = pd.read_excel(excel_data)

df.head()

In [ ]:
# Save the dataset to file directory 
df.to_csv('Customer_understanding_kmeans.csv', index=False)

In [ ]:
df = pd.read_csv('Customer_understanding_kmeans.csv')

In [ ]:
df.head()

In [ ]:
# Drop the column with unnamed
df = df.drop(columns=['Unnamed: 0'])

# 3. Dataset Inspection 

In [ ]:
# Dataset inspection 
df.head()

In [ ]:
# Dataset Info 
df.info()

In [ ]:
# Define function that performs dataset inspection 
import pandas as pd

def data_inspection(df):
    """
    Performs a professional-grade inspection of the dataframe.
    """
    print("=== Dataset Overview ===")
    print(f"Total Rows: {df.shape[0]}")
    print(f"Total Columns: {df.shape[1]}")
    print(f"Duplicate Rows: {df.duplicated().sum()}")
    print("-" * 30)
    
    # Create a summary table for columns
    inspection_df = pd.DataFrame({
        'Data Type': df.dtypes,
        'Missing Values': df.isnull().sum(),
        'Missing %': (df.isnull().sum() / len(df) * 100).round(2),
        'Unique Values': df.nunique()
    })
    
    print("=== Column Statistics ===")
    display(inspection_df) # Use display() for a nice table in Jupyter
    
    print("\n=== Sample Data (First 3 Rows) ===")
    display(df.head(3))

data_inspection(df)

The invoice date column is not in the correct datatype and I has to be changed to a date time format 

In [ ]:
# Change invoice date to date time format 
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [ ]:
# Check the conversion 
df['InvoiceDate'].dtypes

Another column that needs converstion is the invoice column which should be the primary key and should be in an integer format since it is a numerical value . 

Now to inspect high level Statistics 

In [ ]:
# high-level statistics for numerical features 
print(df.describe().T)

Negative Quantity (-9600): These could represent cancelled orders or returns. Since K-Means clustering (RFM) calculates "Total Spend," keeping these as-is might distort a customer's value.

Negative Price (-53594.36): This is usually a "bad debt" adjustment or data entry error. A product cannot have a negative price in a standard retail transaction.

Customer ID Missing (~108,000 rows): Since your goal is Customer Segmentation, you cannot cluster transactions that don't have a specific ID assigned to them.

Skewed Data: Look at Quantity: The 75th percentile is 10, but the max is 19,152. This indicates massive outliers (likely wholesalers) that will pull your K-Means centroids away from the average user.

In [ ]:
# Inspect object columns 
print(df.describe(include ='O'))

Geographic Dominance: The United Kingdom appears 485,852 times out of 525,461. This means ~92% of your data is from one country.

It could be useful to  focus segmentation on the UK market specifically to avoid "noise" from different shipping costs/currency behaviors in export markets.

Product Variety: There are 4,632 unique stock codes. This is a high-cardinality feature.
It should also be noted that there are 4681 descriptions this doesnt align with the stock codes and should be further investigated in analysis . 

Transaction Volume: There are 28,816 unique invoices. Since the total rows are over 525k, this confirms that each invoice contains multiple line items (average of ~18 items per basket).

# Inspect missing Values 

In [ ]:
# inspect missing data 
df[df['Customer ID'].isna()].head(10)

It can be see that some of these values do not have positive quantity values and could this mean returns of the items. Since there are no customer id values this makes segmentation inneffective and thus conclude that this data is missing at random and the rows should be dropped . 

In [ ]:
# Inspect negative quantities 
df[df['Quantity'] <0].head(20)

Quite a large number of these negative values are occuring from the same invoice and as and were purchased at the same time . The inference could be that these purchases were returned goods . The C preceeding the invoice number indicates a cancellation of the the order . This should be investigated further . 

In [ ]:
# Inspect invoice numbers 
df['Invoice'] =df['Invoice'].astype('str')
# Use regular expression 
df[df['Invoice'].str.match('^\\d{6}$') == False]

It can be seen that 10 209 invoices resulted in cancellations of the purchase . 

In [ ]:
# To see if C is the only unique character 
df['Invoice'].str.replace('[0-9]' ,'', regex=True).unique()

This indicates that there is invoices with the value A 

In [ ]:
df[df['Invoice'].str.startswith("A")]

These are 3 rows that indicate the descriptionis adjust bad debt that have negative pricing . These seem like accounting transactions. 
These values should be removed from the transactions. 

# Inspect Stock code column 

In [ ]:
# Test stock code only contains 5 digit values 
# 1. Convert stockcode column to string
df['StockCode'] = df['StockCode'].astype('str')
# 2. Inspect values where stockcode is not only 5 digit values. 
df[df['StockCode'].str.match('^\\d{5}$') == False]

These transations are valid however it is not know what each stock code means 

In [ ]:
df[(df['StockCode'].str.match('^\\d{5}$') == False) & (df['StockCode'].str.match('^\\d{5}[a-zA-Z]+$') == False)]

This shows that there are stock codes with the name for Postage , discount, and other codes that dont follow the specific pattern . 

In [ ]:
df[(df['StockCode'].str.match('^\\d{5}$') == False) & (df['StockCode'].str.match('^\\d{5}[a-zA-Z]+$') == False)]['StockCode'].unique()

In [ ]:
# To inspect these values 

values_to_inspect = ['POST', 'D', 'DCGS0058', 'DCGS0068', 'DOT', 'M', 'DCGS0004',
       'DCGS0076', 'C2', 'BANK CHARGES', 'DCGS0003', 'TEST001',
       'gift_0001_80', 'DCGS0072', 'gift_0001_20', 'DCGS0044', 'TEST002',
       'gift_0001_10', 'gift_0001_50', 'DCGS0066N', 'gift_0001_30',
       'PADS', 'ADJUST', 'gift_0001_40', 'gift_0001_60', 'gift_0001_70',
       'gift_0001_90', 'DCGSSGIRL', 'DCGS0006', 'DCGS0016', 'DCGS0027',
       'DCGS0036', 'DCGS0039', 'DCGS0060', 'DCGS0056', 'DCGS0059', 'GIFT',
       'DCGSLBOY', 'm', 'DCGS0053', 'DCGS0062', 'DCGS0037', 'DCGSSBOY',
       'DCGSLGIRL', 'S', 'DCGS0069', 'DCGS0070', 'DCGS0075', 'B',
       'DCGS0041', 'ADJUST2', '47503J ', 'C3', 'SP1002', 'AMAZONFEE']


# 4. Data Cleaning 

In [ ]:
# copy dataset for cleaning 
cleaned_df = df.copy()

### Clean Invoice Column 

In [ ]:
cleaned_df

To s

In [ ]:
cleaned_df['Invoice'] = cleaned_df['Invoice'].astype('str')
mask = (
    cleaned_df['Invoice'].str.match('^\\d{6}$') == True 
)

cleaned_df = cleaned_df[mask]

In [ ]:
# Find matches by digits or digits and are followed by letter and pads stockcode
cleaned_df["StockCode"] = cleaned_df['StockCode'].astype('str')
stock_mask = (
    (cleaned_df['StockCode'].str.match('^\\d{5}$') == True)
    | (cleaned_df['StockCode'].str.match('^\\d{5}[a-zA-Z]+$') == True)
    | (cleaned_df['StockCode'].str.match('^PADS$') == True)
)

cleaned_df = cleaned_df[stock_mask]

cleaned_df

In [ ]:
# We must deal with quantities less than zero drop missing values and also 
# Create a filter that removes rows ONLY if BOTH conditions are met

cleaned_df.dropna(subset = ['Customer ID'], inplace = True)

In [ ]:
cleaned_df.describe()

 There are prices that are equal to zero which will not be helpful in our customer analysis 

In [ ]:
cleaned_df[cleaned_df['Price']== 0]

In [ ]:
# filter out price = 0 
cleaned_df = cleaned_df[cleaned_df['Price']>0]

To finalise the cleaning process is to see how much data has been lost 


In [ ]:
# calculate the percentage of remaining data 
pct_data_left = (len(cleaned_df) / len(df))* 100 
pct_data_lost = 100 - pct_data_left
print(f' After the cleaning process there remains {pct_data_left:.2f}% of the original data')
print(f' The cleaning process removed {pct_data_lost:.2f} % of the data' )

# 5. Feature Engineering 

This section will perform the following functions 
1. Sales Totals 
2. Aggregation of Data By Customer ID 
3. Computing Recency Value 
4. Computing Frequency Value 
5. Computing Monetary Value 

In [ ]:
# Create a new column in the cleaned dataframe to find the total amount per sale 
cleaned_df['Sales_total_pounds'] = cleaned_df['Quantity']* cleaned_df['Price']
cleaned_df

Now to create a new dataframe that will be aggregated 

In [ ]:
# Create an aggregated df that groups by Monetary Frequency and Recency value 
aggregated_df = cleaned_df.groupby('Customer ID', as_index = False).agg(
    Monetary_value = ('Sales_total_pounds', "sum"),
    Frequency = ('Invoice', 'nunique'),  # counts the number of unique invoices per customer id 
    Last_invoice_date = ('InvoiceDate', 'max')
    
    
)
aggregated_df.head(5)                                              

In [ ]:
# For recency we have to look at the date the last invoice was made . 
max_invoice_date = aggregated_df['Last_invoice_date'].max()

# Create new column called recency which calculates the days between the invoices
aggregated_df['Recency']= (max_invoice_date - aggregated_df['Last_invoice_date']).dt.days
# Inspection of the dataframe 
aggregated_df.head(5)

Observe the distribution of these features to see if there are outliers that can influence the Kmeans clustering algorithm . 

Monetary value is skewed. THis is the same for Frequecy with outliers in these distributions. 
    
Now we inspect for outliers 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the visual style
sns.set_style("whitegrid")
plt.figure(figsize=(15, 6))

# Plot 1: Monetary Value Boxplot
plt.subplot(1, 3, 1)
sns.boxplot(y=aggregated_df['Monetary_value'], color='skyblue')
plt.title('Monetary Value Outliers', fontsize=14, fontweight='bold')
plt.ylabel('Monetary Value (Pounds)')

# Plot 2: Frequency Boxplot
plt.subplot(1, 3, 2)
sns.boxplot(y=aggregated_df['Frequency'], color='lightgreen')
plt.title('Frequency Outliers', fontsize=14, fontweight='bold')
plt.ylabel('Frequency')

# Plot 3: Recency Boxplot
plt.subplot(1, 3, 3)
sns.boxplot(y=aggregated_df['Recency'], color='salmon')
plt.title('Recency Outliers', fontsize=14, fontweight='bold')
plt.ylabel('Recency (Days)')

plt.tight_layout()
plt.show()

Significant Skewness: Both Monetary Value and Frequency exhibit heavy right-skewness. A small percentage of customers contribute a disproportionately high amount of revenue and transaction volume.

Extreme Outliers: We see customers with spend exceeding £300,000, while the median is significantly lower. These outliers will dominate distance-based algorithms like K-Means.

Strategic Treatment: To ensure a meaningful segmentation, I will apply a Log Transformation to these features. This will "compress" the distance between the outliers and the average customers, making the data more normally distributed and suitable for clustering.

In [ ]:
# separate the monetary outliers using interquartile range 
m_quartile1 =aggregated_df['Monetary_value'].quantile(0.25)
m_quartile3 = aggregated_df['Monetary_value'].quantile(0.75)
m_iqr = m_quartile3 - m_quartile1

# Now to address the outliers 
monetary_outliers_df = aggregated_df[(aggregated_df['Monetary_value'] > (m_quartile3 + 1.5 * m_iqr))].copy()
monetary_outliers_df.describe()

In [ ]:
# separate the monetary outliers using interquartile range 
f_quartile1 =aggregated_df['Frequency'].quantile(0.25)
f_quartile3 = aggregated_df['Frequency'].quantile(0.75)
f_iqr = f_quartile3 - f_quartile1

# Now to address the outliers 
frequency_outliers_df = aggregated_df[(aggregated_df['Frequency'] > (f_quartile3 + 1.5 * f_iqr))].copy()
frequency_outliers_df.describe()

The monetary value outliers and frequency outliers likely will have ovelap 
Ill have to look at the pure monetary and pure frequency outliers and overlapping outliers . 

In [ ]:
# Create new dataframe to filter for values that are not outliers 
non_outliers_df = aggregated_df[(~aggregated_df.index.isin(monetary_outliers_df.index))& (~aggregated_df.index.isin(frequency_outliers_df.index))]
non_outliers_df.describe()

Now to visualise the non outlier data vs the outlier data 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the visual style
sns.set_style("whitegrid")

# Create a large figure to hold all 6 plots
plt.figure(figsize=(15, 10))

# --- TOP ROW: ORIGINAL DATA WITH EXTREME OUTLIERS ---
plt.subplot(2, 3, 1)
sns.boxplot(y=aggregated_df['Monetary_value'], color='skyblue')
plt.title('Monetary Value Outliers')

plt.subplot(2, 3, 2)
sns.boxplot(y=aggregated_df['Frequency'], color='lightgreen')
plt.title('Frequency Outliers')

plt.subplot(2, 3, 3)
sns.boxplot(y=aggregated_df['Recency'], color='salmon')
plt.title('Recency Outliers')

# --- BOTTOM ROW: CLEANED DATA (NON-OUTLIERS) ---
plt.subplot(2, 3, 4)
sns.boxplot(y=non_outliers_df['Monetary_value'], color='skyblue')
plt.title('Monetary Value Non Outliers Boxplot')

plt.subplot(2, 3, 5)
sns.boxplot(y=non_outliers_df['Frequency'], color='lightgreen')
plt.title('Frequency Non Outliers Boxplot')

plt.subplot(2, 3, 6)
sns.boxplot(y=non_outliers_df['Recency'], color='salmon')
plt.title('Recency Non Outliers Boxplot')

plt.tight_layout()
plt.show()

After removing extreme outliers (using a method like the Interquartile Range or IQR), you can finally see the "box" and "whiskers" for the majority of your customers. For example, the Monetary Value now focuses on the £0 - £3,500 range, allowing the model to see differences between "normal" customers

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')
scatter = ax.scatter(non_outliers_df['Monetary_value'], non_outliers_df['Frequency'], non_outliers_df['Recency']) 
ax.set_xlabel('Monetary Value')
ax.set_ylabel('Frequency Outliers')
ax.set_zlabel('Recency')
ax.set_title('3D Scatter Plot of Customer Data') 
plt.show()


We notice the data is in different scales . This is an indication of the need to use a scaler since the K means algorithm is sensitive to scaling . 
The algorithm should treat each feature with equal weight. 

# Scaling the data

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(non_outliers_df[['Monetary_value', 'Frequency', 'Recency']])
scaled_data

In [ ]:
scaled_data_df = pd.DataFrame(scaled_data, columns = ['Monetary_value', 'Frequency', 'Recency'], index = non_outliers_df.index)
scaled_data_df.head()

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')
scatter = ax.scatter(scaled_data_df['Monetary_value'], scaled_data_df['Frequency'], scaled_data_df['Recency']) 
ax.set_xlabel('Monetary Value')
ax.set_ylabel('Frequency ')
ax.set_zlabel('Recency')
ax.set_title('3D Scatter Plot of Customer Data Scaled') 
plt.show()


The data is now on comparable scales and is on the same distribution 

# 6. KMeans Clustering 

To determine the optimal clusters for this K means algorithm I shall utilise the Elbow method. 

In [ ]:
max_k = 12

inertia = [] 
silhouette_scores = []

k_values = range(2, max_k+1)
for k in k_values:
    
    # fit predict kmeans clustering 
    kmeans = KMeans(n_clusters=k, random_state=42, max_iter=1000)
     
    cluster_labels = kmeans.fit_predict(scaled_data_df)
    
    sil_score = silhouette_score(scaled_data_df, cluster_labels)
    # append to lists 
    inertia.append(kmeans.inertia_)
    silhouette_scores.append(sil_score)
 
# plotting sihouette scores and elbow method 

plt.figure(figsize=(14, 6))

# plotting elbow method subplot
plt.subplot(1, 2, 1)
plt.plot(k_values, inertia, marker='o')
plt.xticks(k_values, k_values)
plt.xlabel('Number of Clusters')
plt.ylabel('Intertia')
plt.title('KMeans Inertia for Different k values', fontweight='bold')

# plotting silhouette subplot
plt.subplot(1,2,2)
plt.plot(k_values, silhouette_scores, marker='o', color = 'red')
plt.title('Silhouette Scores for Different k values', fontweight='bold')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.xticks(k_values)
plt.grid(True)

plt.show()
    
    

The inertia value rapidly decreases and we look for where the gradient decreases . The elbow has a gradient decrease between 4 and 5 clusters . To choose betweeen the two clusters I shall use the Silhouette score . The higher score is what we want between the two values and the curve on the right indicates that the cluster of 4 has a better score. In conclusion I shall utilise a k of 4 clusters. 

### Final Model 

In [ ]:
# Fitting the model with the correct parameters 
kmeans_final = KMeans(n_clusters=4, random_state=42, max_iter=1000) 

cluster_labels = kmeans_final.fit_predict(scaled_data_df)

cluster_labels

In [ ]:
# Add clusters to original non outliers dataframe 
# The syntax is .loc[all_rows, 'new_column_name']
non_outliers_df.loc[:, 'Cluster'] = cluster_labels
non_outliers_df

### Now to visualise the different clusters on a 3d scatterplot 

In [ ]:
# 1. Define  professional color palette for the segments
# Using standard Hex codes for consistency
cluster_colors = {
    0: '#1f77b4',  # Blue
    1: '#ff7f0e',  # Orange
    2: '#2ca02c',  # Green
    3: '#d62728'   # Red
}

# 2. Map the clusters to their respective colors
colors = non_outliers_df['Cluster'].map(cluster_colors)

# 3. Initialize the 3D plot
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(projection='3d')

# 4. Create the scatter plot
# Using your specific RFM columns from the aggregated dataframe
scatter = ax.scatter(
    non_outliers_df['Monetary_value'],
    non_outliers_df['Frequency'],
    non_outliers_df['Recency'],
    c=colors,           # Use the mapped solid colors
    marker='o',         # Spherical markers
    alpha=0.6,          # Slight transparency to see overlapping points
    edgecolors='w',     # White edges for better point separation
    s=60                # Marker size
)

# 5. Set labels and title
ax.set_xlabel('Monetary Value (£)', labelpad=10, fontweight='bold')
ax.set_ylabel('Frequency (Orders)', labelpad=10, fontweight='bold')
ax.set_zlabel('Recency (Days)', labelpad=10, fontweight='bold')
ax.set_title('3D Customer Segmentation: RFM Clusters', fontsize=16, pad=20)

# 6. Add a legend to make it professional
from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], marker='o', color='w', label=f'Cluster {i}',
                          markerfacecolor=color, markersize=10) 
                   for i, color in cluster_colors.items()]
ax.legend(handles=legend_elements, title="Customer Segments", loc='upper left')

plt.tight_layout()
plt.show()

Violin plots to better understand the distribution of the individual clusters. 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure Cluster is integer to match palette keys
non_outliers_df['Cluster'] = non_outliers_df['Cluster'].astype(int)

# Professional color palette
cluster_palette = {0: '#1f77b4', 1: '#ff7f0e', 2: '#2ca02c', 3: '#d62728'}

plt.figure(figsize=(12, 18))

# Plot 1: Monetary Value by Cluster
plt.subplot(3, 1, 1)
sns.violinplot(
    x='Cluster', 
    y='Monetary_value', 
    data=non_outliers_df, 
    palette=cluster_palette, 
    hue='Cluster',      # Fix: Assigning the x variable to hue
    legend=False        # Fix: Removing the redundant legend
)
plt.title('Monetary Value by Cluster', fontsize=14, fontweight='bold')

# Plot 2: Frequency by Cluster
plt.subplot(3, 1, 2)
sns.violinplot(
    x='Cluster', 
    y='Frequency', 
    data=non_outliers_df, 
    palette=cluster_palette, 
    hue='Cluster', 
    legend=False
)
plt.title('Frequency by Cluster', fontsize=14, fontweight='bold')

# Plot 3: Recency by Cluster
plt.subplot(3, 1, 3)
sns.violinplot(
    x='Cluster', 
    y='Recency', 
    data=non_outliers_df, 
    palette=cluster_palette, 
    hue='Cluster', 
    legend=False
)
plt.title('Recency by Cluster', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

The violin plots show the distribution of Recency, Frequency, and Monetary value for each cluster. Here is the interpretation of that data based on your results:

Monetary Value by Cluster: Identifies "Big Spenders" (Cluster 3) versus those who make smaller, occasional purchases (Cluster 1).

Frequency by Cluster: Helps distinguish loyal, repeat customers (Cluster 3) from one-time or infrequent shoppers.

Recency by Cluster: Crucial for identifying "at-risk" customers (Cluster 1) who haven't visited in a long time compared to recent shoppers (Clusters 0, 2, and 3).

---

##  Cluster 3: The Champions (The High-Value Segment)

These are your best customers. They represent the "Gold Standard" of your retail business.

* **Data Profile:** Highest **Monetary Value** (they spend the most) and highest **Frequency** (they shop the most often). Their **Recency** is very low, meaning they have made a purchase very recently.
* **Psychology:** They have a high degree of trust in the brand and likely consider you their "go-to" for gift-ware.
* ** Strategy:** "Maintain and Reward." Do not spam them with generic discounts. Instead, offer them exclusive early access to new collections or a dedicated account manager.

##  Cluster 0: Potential Loyalists (The Growth Segment)

These are your rising stars. They are active and spending well, but they haven't reached "Champion" status yet.

* **Data Profile:** Low **Recency** (recently active) and moderate-to-high **Monetary Value**. Their **Frequency** is stable but lower than Cluster 3.
* **Psychology:** They are currently engaged with your brand and are in the "habit-forming" stage of their customer journey.
* ** Strategy:** "Upsell and Cross-sell." Use their recent purchase history to recommend complementary products. The goal is to increase their "Frequency" to move them into Cluster 3. These may also benefit from loyalty programs 

##  Cluster 2: New or Occasional Shoppers (The Entry Segment)

This group is often the largest in terms of raw customer count.

* **Data Profile:** Very low **Frequency** (often just 1 or 2 orders) and low **Monetary Value**. However, their **Recency** is low, meaning they have visited recently.
* **Psychology:** They are either brand new to the store or only shop for specific, infrequent occasions.
* ** Strategy:** "Nurture and Educate." Focus on "Welcome" email sequences. Provide them with "Social Proof" (reviews/testimonials) to build the trust necessary for a second or third purchase.

## Cluster 1: At-Risk / Hibernating (The Churn Segment)

This is your "Warning" group. These customers were once active but have gone cold.

* **Data Profile:** Highest **Recency** (it has been a long time since their last purchase). Their **Monetary** and **Frequency** values are typically low to moderate.
* **Psychology:** They may have switched to a competitor, had a bad experience, or simply forgotten about the brand.
* ** Strategy:** "Re-activate or Let Go." This is the only group where aggressive discounting is justified. Send a "We Miss You" campaign with a 20% discount to see if they can be moved back into the "Potential Loyalist" group.

---


# Monetary and Frequency Outliers 

# --- STEP: Outlier Categorization & Segment Isolation ---

In [ ]:

# We identify the overlap to avoid double-counting customers who are outliers in both metrics
overlap_indices = monetary_outliers_df.index.intersection(frequency_outliers_df.index)

# Isolate customers who are outliers ONLY in their spending (Monetary)
monetary_only_outliers = monetary_outliers_df.drop(overlap_indices)

# Isolate customers who are outliers ONLY in their purchase regularity (Frequency)
frequency_only_outliers = frequency_outliers_df.drop(overlap_indices)

# Identify the customers who are extreme outliers in both Spend and Frequency
monetary_and_frequency_outliers = monetary_outliers_df.loc[overlap_indices]


# --- Manual Labeling Strategy ---

In [ ]:

# We assign negative cluster IDs to distinguish these from the K-Means generated clusters (0, 1, 2, 3).
# This allows us to keep the extreme data points accessible for separate high-value marketing analysis.
monetary_only_outliers['Cluster'] = -1             # Flag for extreme high-spenders
frequency_only_outliers['Cluster'] = -2            # Flag for extreme high-frequency shoppers
monetary_and_frequency_outliers['Cluster'] = -3    # Flag for elite 'top-tier' outliers (both)

# Unified outlier repository for specialized business reporting
outlier_cluster_df = pd.concat([
    monetary_only_outliers, 
    frequency_only_outliers, 
    monetary_and_frequency_outliers
])
outlier_cluster_df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Define a high-contrast palette for the outliers to distinguish them from standard clusters
# -1: Monetary Outliers, -2: Frequency Outliers, -3: Both
cluster_colors2 = {-1: '#9467bd', -2: '#8c564b', -3: '#e377c2'}

plt.figure(figsize=(12, 18))

# --- Plot 1: Monetary Value Distribution in Outliers ---
plt.subplot(3, 1, 1)
# Ensure you use the correct column name from your dataframe (e.g., 'Monetary_value' or 'MonetaryValue')
sns.violinplot(
    x='Cluster', 
    y='Monetary_value', 
    data=outlier_cluster_df, 
    palette=cluster_colors2,
    hue='Cluster',
    legend=False
)
plt.title('Monetary Value By Cluster', fontsize=14, fontweight='bold')
plt.ylabel('Monetary Value (£)')

# --- Plot 2: Frequency Distribution in Outliers ---
plt.subplot(3, 1, 2)
sns.violinplot(
    x='Cluster', 
    y='Frequency', 
    data=outlier_cluster_df, 
    palette=cluster_colors2,
    hue='Cluster',
    legend=False
)
plt.title('Frequency Value By Cluster', fontsize=14, fontweight='bold')
plt.ylabel('Frequency (Orders)')

# --- Plot 3: Recency Distribution in Outliers ---
plt.subplot(3, 1, 3)
sns.violinplot(
    x='Cluster', 
    y='Recency', 
    data=outlier_cluster_df, 
    palette=cluster_colors2,
    hue='Cluster',
    legend=False
)
plt.title('Recency Value By Cluster', fontsize=14, fontweight='bold')
plt.ylabel('Recency By Cluster')

plt.tight_layout()
plt.show()

###  Strategic Outlier Summary

The analysis isolated three distinct "Manual Clusters" for customers who exceeded the standard interquartile range (IQR) for spend or frequency:

* **Cluster -1 (Monetary Outliers) — PAMPER**:
* **Characteristics**: These are high spenders who make large but infrequent purchases.
* **Potential Strategy**: Focus on maintaining loyalty through personalized offers or luxury services that cater specifically to their high spending capacity.


* **Cluster -2 (Frequency Outliers) — UPSELL**:
* **Characteristics**: These are consistently engaged, frequent buyers who spend less per individual purchase.
* **Potential Strategy**: Implement loyalty programs or bundle deals to encourage higher spending per visit, leveraging their existing high engagement.


* **Cluster -3 (Monetary & Frequency Outliers) — DELIGHT**:
* **Characteristics**: Representing the most valuable "Elite" outliers, these customers exhibit both extreme spending and extreme purchase frequency.
* **Potential Strategy**: Develop dedicated VIP programs or exclusive offers to maintain their high-tier loyalty and encourage continued engagement.



In [ ]:
# Now to combine all our clusters into a single dataframe 
#1. Create a dictionary of cluster labels 

cluster_labels_final = { 
    0: 'Core Regulars',    # Was Retain
    1: 'Win-back Opportunity', # Was Re-engage
    2: 'Promising Leads',  # Was Nurture
    3: 'Loyalty Members',  # Was Reward
   -1:'VIP Elites',       # Was Pamper (Cluster -1)
   -2: 'High-Volume Growth', # Was Upsell (Cluster -2)
   -3: 'Platinum Whales'   # Was Delight (Cluster -3)
}

In [ ]:
final_cluster_df = pd.concat([non_outliers_df, outlier_cluster_df])
final_cluster_df

In [ ]:
# Now to map the specific clusters into the dataframe 
final_cluster_df['Cluster_label'] = final_cluster_df['Cluster'].map(cluster_labels_final)
final_cluster_df


## Visualise results 



In [ ]:
# Calculate counts and feature means
cluster_counts = final_cluster_df['Cluster_label'].value_counts()
feature_means = final_cluster_df.groupby('Cluster_label')[['Recency', 'Frequency', 'Monetary_value']].mean()

# Initialize the plot
fig, ax1 = plt.subplots(figsize=(14, 8))

# 1. Bar Plot: Number of Customers per Segment
sns.barplot(x=cluster_counts.index, y=cluster_counts.values, ax=ax1, palette='Spectral', hue=cluster_counts.index, legend=False)
ax1.set_ylabel('Number of Customers', color='b', fontsize=12, fontweight='bold')
ax1.set_title('Final Customer Segment Distribution & Average Metrics', fontsize=16, fontweight='bold', pad=20)
plt.xticks(rotation=45, ha='right')

# 2. Line Plot: Feature Means (Dual Axis)
ax2 = ax1.twinx()
sns.lineplot(data=feature_means, ax=ax2, palette='crest', marker='o', linewidth=2.5, markersize=10)
ax2.set_ylabel('Average Metric Value', color='g', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

This dual-axis chart effectively communicates the balance between customer volume and business value.

Customer Volume (Bars): The segments on the left represent the majority of the customer base. "Promising Leads" is the largest group, followed by "Core Regulars" and "Win-back Opportunities." This indicates a large pool of potential value that requires nurturing and reactivation.

Average Metric Value (Line Plots): The lines represent the average Monetary Value, Frequency, and Recency for each group.

Platinum Whales: While this group has a smaller customer count, the blue dashed line spikes significantly, showing that these individuals contribute the highest individual monetary value to the business.

VIP Elites & High-Volume Growth: These segments also show elevated monetary contributions compared to the core retail groups, despite their smaller population sizes.

Frequency and Recency: These metrics remain relatively stable across most segments, but the sharp contrast in Monetary Value (blue line) clearly distinguishes the high-value "Whales" from the general population.

## Project Conclusion: Customer Segmentation of Online Retail II

This project successfully transformed a raw dataset of over one million transactions into a strategic roadmap for business growth. By leveraging **RFM Analysis** and **K-Means Clustering**, we moved beyond basic data exploration to discover actionable customer personas.

---

###  Key Achievements

* **Data Integrity & Preprocessing:** Addressed significant data quality issues by handling missing Customer IDs and filtering out transaction anomalies like returns (negative quantities) and debt adjustments (negative prices).
* **Strategic Outlier Management:** Rather than simply deleting extreme data points, "Whale" customers were isolated into manual segments (Monetary, Frequency, and Dual Outliers). This protected the integrity of the K-Means model while preserving the highest-value accounts for specialized B2B handling.
* **Optimal Segment Discovery:** Utilizing the Elbow Method and Silhouette Scores, we identified  as the mathematically optimal number of clusters for the core customer base.
* **Multidimensional Insight:** Through 3D Scatter Plots and Violin Plots, we validated that each cluster represents a distinct behavioral pattern across Recency, Frequency, and Monetary value.

---

###  Final Business Impact

The true value of this analysis lies in its application to marketing and operations:

* **Precision Marketing:** The business can now replace "mass-blasting" emails with targeted campaigns, such as **Loyalty Rewards for Champions** (Cluster 3) and **Reactivation Discounts for At-Risk customers** (Cluster 1).
* **Resource Allocation:** By identifying **Potential Loyalists** (Cluster 0), the marketing team can focus acquisition and upselling efforts on the group most likely to become the next generation of high-value shoppers.
* **VIP Retention:** The separate profiling of **Dual Outliers** ensures that the top 1% of revenue-generating accounts receive the high-touch service or wholesale pricing required to prevent churn.


## Project Conclusion: Customer Segmentation of Online Retail II

This project successfully transformed over one million raw transactions from the **Online Retail II** dataset into a high-impact, customer-centric segmentation strategy. By combining unsupervised machine learning (-Means) with advanced outlier profiling, I identified seven distinct customer groups that enable data-driven marketing decisions.

---

###  Key Methodological Achievements

* **Data Integrity:** Implemented a robust cleaning pipeline to handle negative quantities (returns), missing customer identifiers, and non-product stock codes like "POSTAGE," ensuring the final analysis was based on genuine purchase behavior.
* **Strategic Outlier Management:** Utilizing the Interquartile Range (IQR) method, I isolated extreme "Whales" (Clusters -1, -2, and -3) before training the model. This prevented the centroids from distorting and allowed for a highly accurate -Means model for the core retail population.
* **Mathematical Validation:** Optimized the clustering model () through the **Silhouette Score** and **Elbow Method**, validating that the resulting segments were statistically significant and well-separated in 3D space.

---

###  Business Impact and Behavioral Insights

#### 1. The Core Retail Segments (Clusters 0 to 3)

The analysis identified four key archetypes within the standard customer base:

* **Champions (Cluster 3):** The highest spenders and most frequent visitors. **Strategy:** VIP loyalty rewards.
* **Potential Loyalists (Cluster 0):** High recency and growing spend. **Strategy:** Cross-selling to increase purchase frequency.
* **Recent/Occasional (Cluster 2):** New shoppers with low frequency but high engagement. **Strategy:** Personalized welcome sequences.
* **At-Risk (Cluster 1):** Customers who have not visited in over 6 months. **Strategy:** Re-engagement campaigns with win-back discounts.

#### 2. The Elite Outlier Profiles (Clusters -1 to -3)

By profiling the outliers separately, I uncovered critical B2B and "Super-Shopper" segments:

* **PAMPER (Cluster -1):** High-ticket, infrequent spenders. **Focus:** Maintaining loyalty with luxury service.
* **UPSELL (Cluster -2):** Highly frequent buyers with **low monetary value** per order. **Focus:** Encouraging bundle deals and larger orders to optimize shipping costs.
* **DELIGHT (Cluster -3):** The most valuable "Dual Outliers" with extreme spend and frequency. **Focus:** Dedicated VIP account management.

---

###  Final Summary

This analysis proves that a one-size-fits-all marketing approach is inefficient. By leveraging **Violin Plots** to understand data density and **-Means** to define behavior, this project provides the business with a roadmap to maximize **Customer Lifetime Value (CLV)**. The distinction between "Whales" and "Retail Shoppers" ensures that marketing resources are allocated where they will generate the highest return on investment.


###  Technical Stack

`Python` | `Pandas` | `Scikit-Learn (K-Means)` | `Matplotlib` | `Seaborn` | `RFM Feature Engineering`

---
